In [1]:
import torch

# net = get_net(name)
# ckpt = torch.load('assets/int8_vit_base_patch16_224.pth')
ckpt = torch.load('results/deit_tiny_checkpoint.pth.tar')
ckpt

OrderedDict([('cls_token',
              tensor([[[ 2.1779e-02, -4.7433e-02, -6.9587e-01,  1.1539e-01,  3.2108e-01,
                        -1.0154e+00, -1.5996e-03,  4.0718e-01, -3.4928e-01, -1.0791e+00,
                         3.4897e-01, -8.7497e-02, -2.0162e-01,  3.8870e-01, -1.9678e-01,
                        -2.7191e-01, -6.0878e-01, -2.9874e-01, -1.7185e-01, -1.7439e-01,
                         1.0177e+00,  6.9287e+00,  8.7355e-02, -6.4623e-01,  6.5064e-01,
                        -4.3848e-01, -7.7215e-02,  1.6832e-01,  8.6586e-02, -4.6247e-01,
                        -2.8964e-01,  2.2390e-01,  3.6265e-01,  2.0397e-01, -5.5472e-01,
                        -6.3274e-01,  8.1991e-02,  4.2451e-01, -8.1754e-01, -1.4588e-02,
                        -6.2289e-01,  1.2201e-01, -4.7010e-01,  5.5709e-01, -1.9731e-01,
                        -1.8387e-01, -1.4328e-01,  2.6637e-01, -3.7855e-01,  1.5326e-01,
                        -4.7521e-01,  8.1509e-01,  1.7036e-02,  1.7090e-02, -5.2690

In [4]:

a = torch.tensor(0.22)
print(a.size())

b = torch.tensor([a])
print(b.size())

torch.Size([])
torch.Size([1])


In [ ]:
import argparse
import os
import time
import math
import logging
import numpy as np

import torch
import torch.nn as nn
from pathlib import Path

from timm.data import Mixup
from timm.models import create_model
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy
from timm.scheduler import create_scheduler
from timm.optim import create_optimizer
from timm.utils import NativeScaler, get_state_dict, ModelEma, accuracy

from models import *
from utils import *

def str2model(name):
    d = {'deit_tiny': deit_tiny_patch16_224,
         'deit_small': deit_small_patch16_224,
         'deit_base': deit_base_patch16_224,
         'swin_tiny': swin_tiny_patch4_window7_224,
         'swin_small': swin_small_patch4_window7_224,
         'swin_base': swin_base_patch4_window7_224,
         'vit_base': vit_base_patch16_224,
         'vit_large': vit_large_patch16_224,
         }
    print('Model: %s' % d[name].__name__)
    return d[name]

# Dataset
train_loader, val_loader = dataloader(args)

# Model
model = str2model(args.model)(pretrained=True,
                              num_classes=args.nb_classes,
                              drop_rate=args.drop,
                              drop_path_rate=args.drop_path)
model.to(device)